In [ ]:
!pip install aitextgen

     |████████████████████████████████| 572 kB 5.1 MB/s 
     |████████████████████████████████| 3.1 MB 50.9 MB/s 
     |████████████████████████████████| 87 kB 8.9 MB/s 
     |████████████████████████████████| 524 kB 59.2 MB/s 
     |████████████████████████████████| 329 kB 70.1 MB/s 
     |████████████████████████████████| 596 kB 67.8 MB/s 
     |████████████████████████████████| 829 kB 68.0 MB/s 
     |████████████████████████████████| 132 kB 65.5 MB/s 
     |████████████████████████████████| 1.1 MB 49.7 MB/s 
     |████████████████████████████████| 61 kB 579 kB/s 
     |████████████████████████████████| 3.3 MB 26.8 MB/s 
     |████████████████████████████████| 895 kB 21.2 MB/s 
     |████████████████████████████████| 160 kB 71.7 MB/s 
     |████████████████████████████████| 271 kB 72.0 MB/s 
     |████████████████████████████████| 192 kB 63.0 MB/s 
  Created wheel for aitextgen: filename=aitextgen-0.5.2-py3-none-any.whl size=575905 sha256=0ff3d4fbdadf40c77f9219774b8a06f1bd29fe2ef5e

In [ ]:
!pip install bleu

  Created wheel for bleu: filename=bleu-0.3-py3-none-any.whl size=5801 sha256=c58b570ad6e8f0c7a44dac90496ce79696a97729041128127ce5a961a6bf4ae1
  Stored in directory: /root/.cache/pip/wheels/90/95/f7/3b9dd43fae308b83b018fb3c1b8647d622b3401c23a7ebc41e
Successfully built bleu


In [ ]:
import json

api_token = {"username":"courageekoh","key":"83bc06bfc6490721d52067847e849603"}

!mkdir .kaggle
!mkdir ~/.kaggle

with open('/content/kaggle.json', 'w') as file:
  json.dump(api_token, file)

!cp /content/kaggle.json /root/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!pip install kaggle
!kaggle --version

Kaggle API 1.5.4


In [ ]:
!kaggle datasets download -d dehbaiyor/gpt-finetuned-model

 96% 234M/245M [00:01<00:00, 176MB/s]
100% 245M/245M [00:01<00:00, 156MB/s]


In [ ]:
!unzip gpt-finetuned-model.zip -d data 

Archive:  gpt-finetuned-model.zip
  inflating: data/chess_task2_gpt_finetune_model_v1/chess_task2_gpt_finetune_model_v1/aitextgen.tokenizer.json  
  inflating: data/chess_task2_gpt_finetune_model_v1/chess_task2_gpt_finetune_model_v1/config.json  
  inflating: data/chess_task2_gpt_finetune_model_v1/chess_task2_gpt_finetune_model_v1/pytorch_model.bin  
  inflating: data/gpt_chess_task1_model_v1/gpt_chess_task1_model_v1/aitextgen.tokenizer.json  
  inflating: data/gpt_chess_task1_model_v1/gpt_chess_task1_model_v1/config.json  
  inflating: data/gpt_chess_task1_model_v1/gpt_chess_task1_model_v1/pytorch_model.bin  
  inflating: data/gpt_code_task1_model_v2/gpt_finetune_task1_model_v2/config.json  
  inflating: data/gpt_code_task1_model_v2/gpt_finetune_task1_model_v2/pytorch_model.bin  
  inflating: data/gpt_code_task2_modelv1/gpt_code_task2_modelv1/aitextgen.tokenizer.json  
  inflating: data/gpt_code_task2_modelv1/gpt_code_task2_modelv1/config.json  
  inflating: data/gpt_code_task2_modelv

# Analysis for Code to Text Performance with GPT model

In [ ]:
import pandas as pd
import numpy as np
from aitextgen import aitextgen
from bleu import multi_list_bleu
from tqdm.notebook import tqdm
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
test = pd.read_csv('test_data.csv')

In [ ]:
test.head()

,input,target
0,"def sina_xml_to_url_list(xml_data):\n """"""st...",str->list\n Convert XML to URL List.\n F...
1,"def dailymotion_download(url, output_dir='.', ...",Downloads Dailymotion videos by URL.
2,"def sina_download(url, output_dir='.', merge=T...",Downloads Sina videos by URL.
3,"def sprint(text, *colors):\n """"""Format text...",Format text with color or other effects into A...
4,"def print_log(text, *colors):\n """"""Print a ...",Print a log message to standard error.


In [ ]:
def clean(col):
    return col.split("\n\n")[0].replace('\n', '').replace('  ', ' ')

In [ ]:
def remove_docstring(col, st):
    return col.replace(st, '')

In [ ]:
test['input'] = test.apply(lambda x: remove_docstring(x['input'], x['target']), axis = 1)

In [ ]:
test.head()

,input,target
0,"def sina_xml_to_url_list(xml_data):\n """"""\n...",str->list\n Convert XML to URL List.\n F...
1,"def dailymotion_download(url, output_dir='.', ...",Downloads Dailymotion videos by URL.
2,"def sina_download(url, output_dir='.', merge=T...",Downloads Sina videos by URL.
3,"def sprint(text, *colors):\n """"""""""""\n re...",Format text with color or other effects into A...
4,"def print_log(text, *colors):\n """"""""""""\n ...",Print a log message to standard error.


In [ ]:
test['input'][1]

'def dailymotion_download(url, output_dir=\'.\', merge=True, info_only=False, **kwargs):\n    """\n    """\n\n    html = get_content(rebuilt_url(url))\n    info = json.loads(match1(html, r\'qualities":({.+?}),"\'))\n    title = match1(html, r\'"video_title"\\s*:\\s*"([^"]+)"\') or \\\n            match1(html, r\'"title"\\s*:\\s*"([^"]+)"\')\n    title = unicodize(title)\n\n    for quality in [\'1080\',\'720\',\'480\',\'380\',\'240\',\'144\',\'auto\']:\n        try:\n            real_url = info[quality][1]["url"]\n            if real_url:\n                break\n        except KeyError:\n            pass\n\n    mime, ext, size = url_info(real_url)\n\n    print_info(site_info, title, mime, size)\n    if not info_only:\n        download_urls([real_url], title, ext, size, output_dir=output_dir, merge=merge)'

In [ ]:
test['target'] = test['target'].apply(clean)

In [ ]:
test['input'][1]

'def dailymotion_download(url, output_dir=\'.\', merge=True, info_only=False, **kwargs):\n    """\n    """\n\n    html = get_content(rebuilt_url(url))\n    info = json.loads(match1(html, r\'qualities":({.+?}),"\'))\n    title = match1(html, r\'"video_title"\\s*:\\s*"([^"]+)"\') or \\\n            match1(html, r\'"title"\\s*:\\s*"([^"]+)"\')\n    title = unicodize(title)\n\n    for quality in [\'1080\',\'720\',\'480\',\'380\',\'240\',\'144\',\'auto\']:\n        try:\n            real_url = info[quality][1]["url"]\n            if real_url:\n                break\n        except KeyError:\n            pass\n\n    mime, ext, size = url_info(real_url)\n\n    print_info(site_info, title, mime, size)\n    if not info_only:\n        download_urls([real_url], title, ext, size, output_dir=output_dir, merge=merge)'

In [ ]:
test['target'][1]

'Downloads Dailymotion videos by URL.'

In [ ]:
test_sample = test[test.input.str.len()< 480]

In [ ]:
eval_sample = test_sample.sample(n = 1000).reset_index().drop(columns = 'index')

In [ ]:
eval_sample['predicted'] = ''
eval_sample['bleu_score'] = ''
eval_sample['input'] = 'summarize: ' + eval_sample['input']

In [ ]:
eval_sample.head()

,input,target,predicted,bleu_score
0,"summarize: def write_comment(fh, comment):\n ...",Writes a comment to the file in Java propertie...,,
1,"summarize: def register_post_execute(self, fun...",Register a function for calling after code exe...,,
2,"summarize: def frames_to_samples(frames, hop_l...",Converts frame indices to audio sample indices.,,
3,"summarize: def partition(condition, collection...",Partitions a list into two based on a condition.,,
4,"summarize: def _get_axis_mode(self, axis):\n ...",will get the axis mode for the current series,,


### Load in model

In [ ]:
from nltk.translate import bleu
from nltk.translate.bleu_score import SmoothingFunction
smoothie = SmoothingFunction().method4

In [ ]:
def calc_smooth_bleu(ref, hyp):
  return bleu([ref], hyp, smoothing_function=smoothie)

In [ ]:
mixed_model_ttc = pd.read_csv('/content/smooth_bleu_ttc.csv', lineterminator='\n')[['input', 'target']]
mixed_model_ttc['predicted_mixed_model'] = ''
mixed_model_ttc['smooth_bleu_score'] = ''

FileNotFoundError: ignored

In [ ]:
mixed_model_ttc.head()

,input,target,predicted_mixed_model,smooth_bleu_score
0,generate: vcf2cytosure CGH file for inidividual.,"def vcf2cytosure(store, institute_id, case_nam...",,
1,generate: Brute force regex based HTML tag par...,"def itertags(html, tag):\n """"""\n \n ""...",,
2,generate: Convert PythonCard font description ...,"def migrate_font(font):\n """"\n if 'faceN...",,
3,generate: Cancel an IO block.,"def cancel(self, block):\n """"""\n ...",,
4,generate: Split a command line's arguments in ...,"def arg_split(s, posix=False):\n """"""\n ""...",,


In [ ]:
ai_code_to_text = aitextgen(model_folder="/content/data/gpt_code_task2_modelv1/gpt_code_task2_modelv1",
               tokenizer_file="/content/data/gpt_code_task2_modelv1/gpt_code_task2_modelv1/aitextgen.tokenizer.json",
               to_gpu=True)

In [ ]:

target = 'Return a list of courses for the passed regid.'

In [ ]:
p = ''

a  = ai_code_to_text.generate(max_length = 512, 
                prompt = p, 
                top_k = 100, 
                top_p  = 0.9, 
                temperature = 0.7,
                return_as_list = True,
                n = 10)
ref = [' '.join(target.split())]
pred = [[' '.join(j.replace(p, '').split())] for j in a]

bleu_scores = []

for k in pred:
  score = calc_smooth_bleu(ref[0], k[0])*100
  bleu_scores.append(score)

prediction = pred[np.argmax(bleu_scores)][0]
bleu_score = max(bleu_scores)

In [ ]:
for i in tqdm(range(len(mixed_model_ttc)), total = len(mixed_model_ttc)):
    p = mixed_model_ttc['input'][i]

    a  = ai_code_to_text.generate(max_length = 512, 
                    prompt = p, 
                    top_k = 100, 
                    top_p  = 0.9, 
                    temperature = 0.7,
                    return_as_list = True,
                    n = 10)
    ref = [' '.join(mixed_model_ttc['target'][i].split())]
    pred = [[' '.join(j.replace(p, '').split())] for j in a]

    bleu_scores = []

    for k in pred:
      score = calc_smooth_bleu(ref[0], k[0])*100
      bleu_scores.append(score)

    mixed_model_ttc['predicted_mixed_model'][i] = pred[np.argmax(bleu_scores)][0]
    mixed_model_ttc['smooth_bleu_score'][i] = max(bleu_scores)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
mixed_model_ttc.head()

,input,target,predicted_mixed_model,smooth_bleu_score
0,generate: vcf2cytosure CGH file for inidividual.,"def vcf2cytosure(store, institute_id, case_nam...","def _create_cgh_to_file(self, cgh): """""" """""" if...",13.0985
1,generate: Brute force regex based HTML tag par...,"def itertags(html, tag):\n """"""\n \n ""...","def _get_tag_tags(tag, tag_tag, inside_tag, ta...",22.2535
2,generate: Convert PythonCard font description ...,"def migrate_font(font):\n """"\n if 'faceN...",. def to_python_card_font_string(python_code):...,17.2724
3,generate: Cancel an IO block.,"def cancel(self, block):\n """"""\n ...","def io_block(self, block): """""" """""" try: respon...",26.7881
4,generate: Split a command line's arguments in ...,"def arg_split(s, posix=False):\n """"""\n ""...","If this is a valid like object, then if one of...",38.5004


In [ ]:
mixed_model_ttc.to_csv('text_to_code.csv', index = False)

In [ ]:
eval_sample['bleu_score'][(eval_sample['bleu_score'] != 0) & (eval_sample['bleu_score'] != -1)].describe()

count     201.00
unique    193.00
top        13.53
freq        2.00
Name: bleu_score, dtype: float64

In [ ]:
eval_sample.to_csv('metrics_ctt.csv', index = False)

# Analysis of model performace for Code to Docstring with Encoder-Decoder Model

In [ ]:
pd.to_numeric(eval_sample['bleu_score'][eval_sample['bleu_score'] != 0]).describe()

count    201.000000
mean      17.905920
std        9.775365
min        0.480000
25%       10.790000
50%       17.760000
75%       23.280000
max       58.140000
Name: bleu_score, dtype: float64

In [ ]:
human_eval = eval_sample.sample(n = 50).reset_index().drop(columns = 'index')

In [ ]:
human_eval

,input,target,predicted,bleu_score
0,"summarize: def get_gravatar_profile_url(email,...",Builds a url to a gravatar profile from an ema...,Get the given email based on the given email b...,12.29
1,"summarize: def save_error(self, data, exceptio...",Saves an error in the error list.,Save the data to the data.,26.76
2,"summarize: def get_allocations(self, prefix=No...",Lists all the allocations.,Retrieve allocations from the json.,22.57
3,"summarize: def save_variable_bigdl(tensors, ta...",Save a variable dictionary to a Java object fi...,"callBigDlFunc(bigdl_type, ""saveTensorDictionar...",0.0
4,summarize: def _get_front_ids_one_at_a_time(on...,Yields one onset front ID at a time until they...,summarize: def _get_front_ids_one_at_a_time(on...,0.0
5,summarize: def advance_robots(self):\n ...,Produces a new game state in which the robots ...,Adds the robots to the robots,0.0
6,"summarize: def REBINDING(self):\n """"""""""...",REBINDING state.,if self.script is not None: self.script.script...,0.0
7,summarize: def _swap_on_miss(partition_result)...,"Given a partition_dict result, if the partitio...","return (return (before, item, after)) Swaps th...",10.49
8,"summarize: def parse_line(self, line):\n ...",Updates the dictionary with a single header line.,"summarize: def parse_line(self, line): """""" """"""...",0.0
9,summarize: def compile_relative_distances(self...,Return load and store distances between accesses.,Perform a left distances to the accesses,0.0


In [ ]:
human_eval.to_csv('human_evaluation_code_to_text.csv', index = False)

In [ ]:
def multibleu(ref, pred):
    return multi_list_bleu([' '.join(ref.split())], [[' '.join(pred.split())]])[0]

In [ ]:
confirmation = pd.read_csv('smooth_bleu2_ctt.csv', lineterminator='\n')

In [ ]:
confirmation.head()

,input,target,predicted,smooth_bleu_score
0,"summarize: def write_comment(fh, comment):\n ...",Writes a comment to the file in Java propertie...,return fh Creates a list of comments for the g...,39.686062
1,"summarize: def register_post_execute(self, fun...",Register a function for calling after code exe...,return self._post_execute[func] Registers a fu...,46.899140
2,"summarize: def frames_to_samples(frames, hop_l...",Converts frame indices to audio sample indices.,Converts a list of samples to another Frames t...,38.740937
3,"summarize: def partition(condition, collection...",Partitions a list into two based on a condition.,Partition a list of indicators to a List.,47.906035
4,"summarize: def _get_axis_mode(self, axis):\n ...",will get the axis mode for the current series,Get the modes of an axis mode for the axis.,58.039008


In [ ]:
confirmation['normal_bleu'] = confirmation.apply(lambda x: multibleu(x['target'], x['predicted']), axis = 1)

In [ ]:
confirmation['normal_bleu'].describe()

count    1000.0
mean       -1.0
std         0.0
min        -1.0
25%        -1.0
50%        -1.0
75%        -1.0
max        -1.0
Name: normal_bleu, dtype: float64

In [ ]:
confirmation['target'][1]

'Register a function for calling after code execution.'

In [ ]:
multi_list_bleu(['Register a function for calling after code execution.'], [['return self._post_execute[func] Registers a function.']])

[-1]

In [ ]:
' '.join(ref.split())

'Writes a comment to the file in Java properties format.'

In [ ]:
human_readable = pd.read_csv('smooth_bleu2_ctt.csv', lineterminator='\n').sample(n = 50).reset_index().drop(columns = 'index')

In [ ]:
human_readable.to_csv('human_evaluation_code_to_text_review.csv', index = False)